In [2]:
import pandas as pd
import numpy as np
#from sklearn.impute import SimpleImputers

In [3]:
# Definimos los archivos CSV
archivo_lluvia = './Dataset_INUMET/CSV/Lluvia.csv'
archivo_temperatura = './Dataset_INUMET/CSV/Temperatura.csv'
archivo_humedad = './Dataset_INUMET/CSV/Humedad.csv'
archivo_viento = './Dataset_INUMET/CSV/VientoCarrasco.csv'
archivo_fenomenos = './Dataset_INUMET/CSV/Fenomenos.csv'

# Los cargamos en dataframes
try:
    lluvia_df = pd.read_csv(archivo_lluvia, sep=';')
    temperatura_df = pd.read_csv(archivo_temperatura, sep=';')
    humedad_df = pd.read_csv(archivo_humedad, sep=';')
    viento_df = pd.read_csv(archivo_viento, sep=';')
    fenomenos_df = pd.read_csv(archivo_fenomenos, sep=';')
except Exception as e:
    print(f"Error al leer el archivo CSV: {e}")
    exit()

In [4]:
## Lluvias
# Nos quedamos solamente con las columnas de la fecha y los mm de lluvia
lluvia_df_subset = lluvia_df.iloc[:, [2, 3]].copy()
lluvia_df_subset.columns = ['fecha', 'mm_lluvia']
# Normalizamos el formato de la fecha (pasamos a pandas datetime64[ns])
lluvia_df_subset['fecha'] = pd.to_datetime(lluvia_df_subset['fecha'], format='%d/%m/%Y')
# Verificamos si hay valores no numéricos en lluvia y los reemplazamos por 0, por lo que comentó el profe
lluvia_df_subset['mm_lluvia'] = pd.to_numeric(lluvia_df_subset['mm_lluvia'], errors='coerce').fillna(0)
# Para los valores nulos de lluvia, nos quedamos con la media
lluvia_df_subset['mm_lluvia'].fillna(lluvia_df_subset['mm_lluvia'].mean())
# Definimos día lluvioso como los días con lluvias mayores a 1mm
umbral_lluvia = 1.0
lluvia_df_subset['dia_lluvioso'] = (lluvia_df_subset['mm_lluvia'] >= umbral_lluvia).astype(int)

# Creamos la clase objetivo "mañana llueve"
lluvia_df_subset['manana_llueve'] = lluvia_df_subset['dia_lluvioso'].shift(-1)
# Eliminamos la última fila que ahora tiene un valor NaN en 'manana_llueve'
lluvia_df_subset = lluvia_df_subset.dropna(subset=['manana_llueve'])
lluvia_df_subset['manana_llueve'] = lluvia_df_subset['manana_llueve'].astype(int)

print(lluvia_df_subset.head(5))
print("----")
print(lluvia_df_subset.dtypes)

       fecha  mm_lluvia  dia_lluvioso  manana_llueve
0 1950-01-01        4.6             1              0
1 1950-01-02        0.0             0              0
2 1950-01-03        0.0             0              0
3 1950-01-04        0.0             0              0
4 1950-01-05        0.0             0              0
----
fecha            datetime64[ns]
mm_lluvia               float64
dia_lluvioso              int64
manana_llueve             int64
dtype: object


In [5]:
## Temperatura
# Nos quedamos solamente con las columnas fecha, Tmax y Tmin de Carrasco
temperatura_df_subset = temperatura_df.iloc[:, :3].copy()
temperatura_df_subset.columns = ['fecha', 'temp_max', 'temp_min']
# Normalizamos el formato de la fecha (pasamos a pandas datetime64[ns])
temperatura_df_subset['fecha'] = pd.to_datetime(temperatura_df_subset['fecha'], format='%d/%m/%Y', errors='coerce')
# Eliminamos filas con fechas inválidas
temperatura_df_subset = temperatura_df_subset.dropna(subset=['fecha'])
# Reemplazamos los valores NaN por la media de cada columna
temperatura_df_subset['temp_max'].fillna(temperatura_df_subset['temp_max'].mean())
temperatura_df_subset['temp_min'].fillna(temperatura_df_subset['temp_min'].mean())
# Calculamos la temperatura promedio diaria
temperatura_df_subset['temp_promedio_dia'] = (temperatura_df_subset['temp_max'] + temperatura_df_subset['temp_min']) / 2
# Calculamos los umbrales para categorizar la temperatura
max_temp = temperatura_df_subset['temp_promedio_dia'].max()
min_temp = temperatura_df_subset['temp_promedio_dia'].min()
umbral_alta = min_temp + (max_temp - min_temp) * 2 / 3
umbral_baja = min_temp + (max_temp - min_temp) * 1 / 3
# Creamos funcion para categorizar la temperatura según umbrales en 'Baja', 'Media' y 'Alta'
def categorizar_temperatura(temp):
    if temp < umbral_baja:
        return 'Baja'
    elif umbral_baja <= temp < umbral_alta:
        return 'Media'
    else:
        return 'Alta'
# La aplicamos al dataframe
temperatura_df_subset['temp_categoria_dia'] = temperatura_df_subset['temp_promedio_dia'].apply(categorizar_temperatura)
# Aseguramos que categoria sea de tipo category de pandas
temperatura_df_subset['temp_categoria_dia'] = temperatura_df_subset['temp_categoria_dia'].astype('category')

print(temperatura_df_subset.head(5))
print("----")
print(temperatura_df_subset.dtypes)

       fecha  temp_max  temp_min  temp_promedio_dia temp_categoria_dia
0 1970-01-01      32.6      21.0              26.80               Alta
1 1970-01-02      33.8      22.0              27.90               Alta
2 1970-01-03      28.1      21.5              24.80               Alta
3 1970-01-04      24.0      18.8              21.40              Media
4 1970-01-05      30.5      20.2              25.35               Alta
----
fecha                 datetime64[ns]
temp_max                     float64
temp_min                     float64
temp_promedio_dia            float64
temp_categoria_dia          category
dtype: object


In [6]:
## Humedad
# Nos quedamos solamente con las columnas de la fecha y la humedad relativa de Carrasco
humedad_df_subset = humedad_df.iloc[:,:2].copy()
humedad_df_subset.columns = ['fecha', 'humedad_relativa']
# Normalizamos el formato de la fecha (pasamos a pandas datetime64[ns]), forzando errores a NaT
humedad_df_subset['fecha'] = pd.to_datetime(humedad_df_subset['fecha'], format='%d/%m/%Y %H:%M', errors='coerce')
# Eliminamos filas con fechas no válidas
humedad_df_subset = humedad_df_subset.dropna(subset=['fecha'])
# Transformamos la fecha a solo fecha (sin hora) y lo devolvemos a datetime64[ns] para ser consistentes
humedad_df_subset['fecha'] = humedad_df_subset['fecha'].dt.date
humedad_df_subset['fecha'] = pd.to_datetime(humedad_df_subset['fecha'])
# Agrupamos por dia y calculamos la humedad relativa promedio diaria
humedad_df_subset = humedad_df_subset.groupby('fecha').agg({'humedad_relativa': 'mean'}).reset_index()
# Calculamos los umbrales para categorizar la humedad relativa en 'Baja', 'Media' y 'Alta'
max_humedad = humedad_df_subset['humedad_relativa'].max()
min_humedad = humedad_df_subset['humedad_relativa'].min()
umbral_alta_hum = min_humedad + (max_humedad - min_humedad) * 2 / 3
umbral_baja_hum = min_humedad + (max_humedad - min_humedad) * 1 / 3
# Creamos funcion para categorizar la humedad relativa en 'Baja', 'Media' y 'Alta'
def categorizar_humedad(hum):
    if hum < umbral_baja_hum:
        return 'Baja'
    elif umbral_baja_hum <= hum < umbral_alta_hum:
        return 'Media'
    else:
        return 'Alta'
# La aplicamos al dataframe
humedad_df_subset['humedad_categoria_dia'] = humedad_df_subset['humedad_relativa'].apply(categorizar_humedad)
# Aseguramos que categoria sea de tipo category de pandas
humedad_df_subset['humedad_categoria_dia'] = humedad_df_subset['humedad_categoria_dia'].astype('category')

print(humedad_df_subset.head(5))
print("----")
print(humedad_df_subset.dtypes)

       fecha  humedad_relativa humedad_categoria_dia
0 1980-01-01         64.857143                 Media
1 1980-01-02         67.428571                 Media
2 1980-01-03         61.714286                 Media
3 1980-01-04         63.285714                 Media
4 1980-01-05         46.142857                  Baja
----
fecha                    datetime64[ns]
humedad_relativa                float64
humedad_categoria_dia          category
dtype: object


In [7]:
## Viento
# Nos quedamos solamente con las columnas de la fecha y la dirección del viento de Carrasco
viento_df_subset = viento_df.iloc[:, :2].copy()
viento_df_subset.columns = ['fecha', 'direccion_viento']
# Normalizamos el formato de la fecha (pasamos a pandas datetime64[ns]), forzando errores a NaT
viento_df_subset['fecha'] = pd.to_datetime(viento_df_subset['fecha'], format='%d/%m/%Y %H:%M', errors='coerce')
# Eliminamos filas con fechas no válidas
viento_df_subset = viento_df_subset.dropna(subset=['fecha'])
# Transformamos la fecha a solo fecha (sin hora) y lo devolvemos a datetime64[ns] para ser consistentes
viento_df_subset['fecha'] = viento_df_subset['fecha'].dt.date
viento_df_subset['fecha'] = pd.to_datetime(viento_df_subset['fecha'])
# Creamos funcion para categorizar la dirección del viento de grados a una de las 8 direcciones
def categorizar_direccion_viento(grados):
    if grados is None or pd.isna(grados):
        return np.nan
    try:
        grados = float(grados)
    except (ValueError, TypeError):
        return np.nan
    dirs = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    ix = round(grados / 45) % 8
    return dirs[ix] 
# La aplicamos al dataframe
viento_df_subset['direccion_viento'] = viento_df_subset['direccion_viento'].apply(categorizar_direccion_viento)
# Aseguramos que categoria sea de tipo category de pandas
viento_df_subset['direccion_viento'] = viento_df_subset['direccion_viento'].astype('category')
# Una vez categorizado, agrupamos por dia y calculamos la dirección del viento predominante diaria
viento_df_subset = viento_df_subset.groupby('fecha').agg({'direccion_viento': lambda x: x.mode()[0] if not x.mode().empty else np.nan}).reset_index()

print(viento_df_subset.head(5))
print("----")
print(viento_df_subset.dtypes)

       fecha direccion_viento
0 1986-07-07               NE
1 1986-07-08               NE
2 1986-07-09                N
3 1986-07-10                N
4 1986-07-11                N
----
fecha               datetime64[ns]
direccion_viento          category
dtype: object


In [8]:
## Fenómenos
# Nos quedamos solamente con las columnas de la fecha y el fenómeno
fenomenos_df_subset = fenomenos_df.iloc[:, :2].copy()
fenomenos_df_subset.columns = ['fecha', 'fenomeno']
# Normalizamos el formato de la fecha (pasamos a pandas datetime64[ns])
fenomenos_df_subset['fecha'] = pd.to_datetime(fenomenos_df_subset['fecha'], format='%d/%m/%Y', errors='coerce')
# Eliminamos filas con fechas no válidas
fenomenos_df_subset = fenomenos_df_subset.dropna(subset=['fecha'])
# Transformamos la fecha a solo fecha (sin hora) y lo devolvemos a datetime64[ns] para ser consistentes
fenomenos_df_subset['fecha'] = fenomenos_df_subset['fecha'].dt.date
fenomenos_df_subset['fecha'] = pd.to_datetime(fenomenos_df_subset['fecha'])
# Agrupamos por dia y marcamos si hubo tormenta con truenos (17) ese dia (1 si hubo, 0 si no)
fenomenos_df_subset['hay_truenos'] = fenomenos_df_subset['fenomeno'].apply(lambda x: 1 if x == 17 else 0)
fenomenos_df_subset = fenomenos_df_subset.groupby('fecha').agg({'hay_truenos': 'max'}).reset_index()

print(fenomenos_df_subset.head(5))
print("----")
print(fenomenos_df_subset.dtypes)

       fecha  hay_truenos
0 2000-01-01            0
1 2000-01-02            0
2 2000-01-03            0
3 2000-01-04            0
4 2000-01-05            0
----
fecha          datetime64[ns]
hay_truenos             int64
dtype: object


In [9]:
## Estación
# Tomamos el dataframe con mayor cantidad de filas (lluvia) como base y nos quedamos con la columna fecha
estacion_df = lluvia_df_subset[['fecha']].copy()
# Creamos funcion para determinar la estación del año según la fecha
def determinar_estacion(fecha):
    mes = fecha.month
    dia = fecha.day
    if (mes == 12 and dia >= 21) or (mes <= 3 and (mes != 3 or dia < 20)):
        return 'Verano'
    elif (mes == 3 and dia >= 20) or (mes <= 6 and (mes != 6 or dia < 21)):
        return 'Otoño'
    elif (mes == 6 and dia >= 21) or (mes <= 9 and (mes != 9 or dia < 22)):
        return 'Invierno'
    else:
        return 'Primavera'
# Creamos la columna estación aplicando la función
estacion_df['estacion'] = estacion_df['fecha'].apply(determinar_estacion)
# Aseguramos que categoria sea de tipo category de pandas
estacion_df['estacion'] = estacion_df['estacion'].astype('category')

print(estacion_df.head(5))
print("----")
print(estacion_df.dtypes)

       fecha estacion
0 1950-01-01   Verano
1 1950-01-02   Verano
2 1950-01-03   Verano
3 1950-01-04   Verano
4 1950-01-05   Verano
----
fecha       datetime64[ns]
estacion          category
dtype: object


In [10]:
## Fase lunar
# Tomamos el dataframe con mayor cantidad de filas (lluvia) como base y nos quedamos con la columna fecha
lunar_df = lluvia_df_subset[['fecha']].copy()
# Creamos funcion para determinar la fase lunar según la fecha
def determinar_fase_lunar(fecha):
    ## SUSTITUIR POR LA BIBLIOTECA QUE EL PROFESOR RECOMENDÓ
    año = fecha.year
    mes = fecha.month
    dia = fecha.day
    if mes < 3:
        año -= 1
        mes += 12
    mes += 1
    c = 365.25 * año
    e = 30.6 * mes
    jd = c + e + dia - 694039.09  # jd es la fecha juliana
    jd /= 29.5305882              # dividir por el ciclo lunar
    b = int(jd)                   # int(jd) es el número de ciclos lunares enteros desde la fecha base
    jd -= b                       # jd es la fracción del ciclo lunar actual
    b = round(jd * 8)             # b es ahora el número de fase lunar (0-8)
    if b >= 8:
        b = 0                      # 0 y 8 son luna nueva
    fases = ['Luna Nueva', 'Cuarto Creciente', 'Luna Llena', 'Cuarto Menguante']
    return fases[b // 2]          # Devolvemos una de las cuatro fases principales
    ##
# Creamos la columna fase lunar aplicando la función
lunar_df['fase_lunar'] = lunar_df['fecha'].apply(determinar_fase_lunar)
# Aseguramos que categoria sea de tipo category de pandas
lunar_df['fase_lunar'] = lunar_df['fase_lunar'].astype('category')

print(lunar_df.head(5))
print("----")
print(lunar_df.dtypes)

       fecha        fase_lunar
0 1950-01-01  Cuarto Creciente
1 1950-01-02        Luna Llena
2 1950-01-03        Luna Llena
3 1950-01-04        Luna Llena
4 1950-01-05        Luna Llena
----
fecha         datetime64[ns]
fase_lunar          category
dtype: object


In [17]:
## Merge de todos los dataframes
dataframes = [lluvia_df_subset, temperatura_df_subset, humedad_df_subset, viento_df_subset, fenomenos_df_subset, estacion_df, lunar_df]
from functools import reduce
df_merged_grande = reduce(lambda left, right: pd.merge(left, right, on='fecha', how='outer'), dataframes)
# Ordenamos por fecha
df_merged_grande = df_merged_grande.sort_values(by='fecha').reset_index(drop=True)

# Nos quedamos solo con las columnas que nos interesan
columnas_interes = ['fecha', 'dia_lluvioso', 'temp_categoria_dia', 'humedad_categoria_dia', 'direccion_viento', 'hay_truenos', 'estacion', 'fase_lunar', 'manana_llueve']
df_merged = df_merged_grande[columnas_interes].copy()
# Corregimos tipos de datos para las columnas con valores binarios
df_merged['dia_lluvioso'] = df_merged['dia_lluvioso'].astype('Int64')
df_merged['hay_truenos'] = df_merged['hay_truenos'].astype('Int64')
df_merged['manana_llueve'] = df_merged['manana_llueve'].astype('Int64')

# print(df_merged.head(50))
# print("----")
# print(df_merged.dtypes)

df_merged

## Según el contador de dias de https://onlinealarmkur.com/date/es/
# Hay 27,617 días entre 01/01/1910 y 11/08/2025 (ambos inclusive), es la misma cantidad de filas que tenemos
# Por lo que no hay días faltantes en el dataset final

,fecha,dia_lluvioso,temp_categoria_dia,humedad_categoria_dia,direccion_viento,hay_truenos,estacion,fase_lunar,manana_llueve
0,1950-01-01,1,NaN,NaN,NaN,<NA>,Verano,Cuarto Creciente,0
1,1950-01-02,0,NaN,NaN,NaN,<NA>,Verano,Luna Llena,0
2,1950-01-03,0,NaN,NaN,NaN,<NA>,Verano,Luna Llena,0
3,1950-01-04,0,NaN,NaN,NaN,<NA>,Verano,Luna Llena,0
4,1950-01-05,0,NaN,NaN,NaN,<NA>,Verano,Luna Llena,0
...,...,...,...,...,...,...,...,...,...
27612,2025-08-07,<NA>,Alta,Alta,S,0,NaN,NaN,<NA>
27613,2025-08-08,<NA>,NaN,Alta,S,0,NaN,NaN,<NA>
27614,2025-08-09,<NA>,NaN,NaN,SE,0,NaN,NaN,<NA>
27615,2025-08-10,<NA>,NaN,NaN,NW,0,NaN,NaN,<NA>
